In [ ]:
# ======================================================
# drive 에서 celeba 폴더에 학습된 모델 넣어주시면 됩니다
#
# 모델 다른거 적용하려면
# model = " ... " 이 부분 파일명 수정해서 실행
# ======================================================

model = "dcgan_front_50_epoch_model.pth"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import torch
import torch.nn as nn
import torchvision.utils as vutils
import matplotlib.pyplot as plt
import numpy as np

ngpu = 1
nz = 100
ngf = 64
nc = 3

In [ ]:
class Generator(nn.Module):
    def __init__(self, ngpu):
        super(Generator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            nn.ConvTranspose2d(nz, ngf*8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf*8),
            nn.ReLU(True),
            nn.ConvTranspose2d(ngf*8, ngf*4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf*4),
            nn.ReLU(True),
            nn.ConvTranspose2d(ngf*4, ngf*2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf*2),
            nn.ReLU(True),
            nn.ConvTranspose2d(ngf*2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            nn.ConvTranspose2d(ngf, nc, 4, 2, 1, bias=False),
            nn.Tanh()
        )

    def forward(self, input):
        return self.main(input)

In [ ]:
device = torch.device("cuda" if (torch.cuda.is_available() and ngpu > 0) else "cpu")
netG = Generator(ngpu).to(device)

drive_root = "/content/drive/MyDrive/celeba"
model_path = os.path.join(drive_root, model)

checkpoint = torch.load(model_path, map_location=device, weights_only=True)

netG.load_state_dict(checkpoint)
netG.eval()

print(f"모델 '{model}' 가중치를 성공적으로 로드했습니다.")

In [ ]:
# =====================
# 이미지 1개 Inference
# =====================

with torch.no_grad():
    noise = torch.randn(1, nz, 1, 1, device=device)
    fake = netG(noise).detach().cpu()

single_image_grid = vutils.make_grid(fake, padding=2, normalize=True)

np_image = np.transpose(single_image_grid, (1,2,0))

plt.figure(figsize=(2,2))
plt.axis("off")
plt.imshow(np_image)
plt.show()

In [ ]:
# ============================================================
# 이미지 N 개 생성 후 저장
# 파일 저장할꺼면 실행하고 아니면 실행 No 생성된
# 이미지들은 inference_imgs 폴더에 저장되고 해당 폴더를
# 기반으로 압축파일 만들어짐, 그거 다운로드하면 됨
# ============================================================

N = 5000
save_path = '/content/inference_imgs'
os.makedirs(save_path, exist_ok=True)

for i in range(N):
    with torch.no_grad():
        noise = torch.randn(1, nz, 1, 1, device=device)
        fake = netG(noise).cpu()

    single_image_grid = vutils.make_grid(fake, padding=2, normalize=True)
    np_image = np.transpose(single_image_grid, (1, 2, 0)).numpy()
    filename = f"fake_{i:04d}.jpg"
    filepath = os.path.join(save_path, filename)
    plt.imsave(filepath, np_image)

    if (i + 1) % 100 == 0:
        print(f"{i + 1}/{N} images saved to {save_path}")

!zip -j -r /content/inference_imgs.zip /content/inference_imgs/*